<a href="https://colab.research.google.com/github/AshishShiv101/LLM_Fundamentals/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open("/content/the-verdict.txt","r",encoding="utf-8") as f:
  raw_text =f.read()
  print("Total number of character:",len(raw_text))
  print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
import re
preprocessed = re.split(r'(--|\s|[,\.:;?_!"()\[\]])', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
print(len(preprocessed))

4519


**Creating Token IDs**

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size= len(all_words)
print(vocab_size)

1154


In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
  print(item)
  if i>= 50:
    break

('!', 0)
('"', 1)
("'", 2)
("'Are", 3)
("'It's", 4)
("'coming'", 5)
("'done'", 6)
("'subject", 7)
("'technique'", 8)
("'way", 9)
('(', 10)
(')', 11)
(',', 12)
('--', 13)
('.', 14)
(':', 15)
(';', 16)
('?', 17)
('A', 18)
('Ah', 19)
('Among', 20)
('And', 21)
('Arrt', 22)
('As', 23)
('At', 24)
('Be', 25)
('Begin', 26)
('Burlington', 27)
('But', 28)
('By', 29)
('Carlo', 30)
('Chicago', 31)
('Claude', 32)
('Come', 33)
('Croft', 34)
('Destroyed', 35)
('Devonshire', 36)
("Don't", 37)
('Dubarry', 38)
('Emperors', 39)
('Florence', 40)
('For', 41)
('Gallery', 42)
('Gideon', 43)
('Gisburn', 44)
("Gisburn's", 45)
('Gisburns', 46)
('Grafton', 47)
('Greek', 48)
('Grindle', 49)
("Grindle's", 50)


In [ ]:
tokenizer.decode(ids)

'" It\'s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
 all_tokens = sorted(list(set(preprocessed)))
 all_tokens.extend(["<endoftext>","<|unk|>"])
 vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:

len(vocab.items())

1156

In [ ]:
tokenizer.encode(text)

[1155, 12, 372, 1148, 651, 995, 17, 1155, 1007, 977, 1003, 744, 1007, 1155, 14]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|unk|> the sunlit terraces of the <|unk|>.'

In [ ]:
pip install tiktoken

**Special Context Token**

In [ ]:
import importlib
import tiktoken


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text =( "Hello, this is a test. <|endoftext|> This is another line.<|endoftext|>And a third line.")

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 428, 318, 257, 1332, 13, 220, 50256, 770, 318, 1194, 1627, 13, 50256, 1870, 257, 2368, 1627, 13]


In [ ]:
strings = tokenizer.decode(integers)
print(strings)

Hello, this is a test. <|endoftext|> This is another line.<|endoftext|>And a third line.


In [61]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int= vocab
    self.int_to_str = { i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'(--|\s|[,\.:;?_!"()\[\]])', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [62]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted,you know,"
          Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 69, 1007, 624, 550, 768, 12, 1148, 618, 12, 1, 81, 14, 44, 871, 1130, 776, 815, 14]


In [60]:
class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int= vocab
    self.int_to_str = { i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'(--|\s|[,\.:;?_!"()\[\]])', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [
        item if item in self.str_to_int
        else "<|unk|>" for item in preprocessed
    ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.:?!"()\'])',r'\1',text)
    return text

In [63]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = "<|endoftext|>".join((text1,text2))

print(text)

Hello, do you like tea?<|endoftext|>In the sunlit terraces of the palace.


**CREATING INPUT_TARGET PAIRS**




In [ ]:
with open("/content/the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
context_size = 4
x = enc_text[:context_size]
y = enc_text[1:context_size+1]


print(f"x : {x}")
print(f"y:     {y}")

x : [40, 367, 2885, 1464]
y:     [367, 2885, 1464, 1807]


In [ ]:
for i in range(1,context_size +1):
  context = enc_text[:i]
  desired = enc_text[i]

  print(context,"---->",desired)

[40] ----> 367
[40, 367] ----> 2885
[40, 367, 2885] ----> 1464
[40, 367, 2885, 1464] ----> 1807


In [ ]:
for i in range(1,context_size +1):
  context = enc_text[:i]
  desired = enc_text[i]

  print(tokenizer.decode(context),"---->",tokenizer.decode([desired]))

I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought


**DATA LOADER**

In [ ]:
 from torch.utils.data import Dataset, DataLoader
 class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids = []
    self.target_ids = []

    token_ids= tokenizer .encode(txt,allowed_special={"<|endoftext|>"})
    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1: i+max_length+1]

      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")

  dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)

  return dataloader


In [ ]:
with open("/content/the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

In [ ]:
import torch
print("PyTorch version:",torch.__version__)
dataloader= create_dataloader_v1(
    raw_text,batch_size=1,max_length= 4,stride=1,shuffle=False
)
data_iter = iter(dataloader)
first_batch= next(data_iter)
print(first_batch)

PyTorch version: 2.6.0+cu124
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
input_ids = torch.tensor([2,3,5,1])

In [ ]:
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


**Positional Embedding**

In [ ]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False
)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)

In [ ]:
print("Token IDs:\n",inputs)
print("\nInputs Shape : \n",inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs Shape : 
 torch.Size([8, 4])


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


**Data PreProcessing**